In [11]:
# coding: utf-8
import numpy as np
import psycopg2
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as pseg
import re
import pickle as pkl
import word2vec
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.cluster import KMeans
jieba.load_userdict("dict.txt.big") 

In [7]:
conn = psycopg2.connect(database="law1", user="datac1", password="datac15543", host="ci.lu.im.ntu.edu.tw", port="5432")
print("Opened database successfully")

cur = conn.cursor()



Opened database successfully


In [36]:
allText = []
ix2Squid = {}
for i in range(11950,11949,-1):
    cur.execute("""SELECT jfull FROM jdata where sqlid IN (Select sqlid FROM jdata_meta WHERE sqlid >= %d and sqlid < %d and cat = 'M') """ % ((i-1)*1000,i*1000))
    rows = cur.fetchall()

    for row in rows:
        #ix2Squid[len(allText)] = row[0]
        allText.append([t for t in jieba.lcut(row[0]) if re.match("^[\u4e00-\u9fa5]",t)] )
        
    print(i,len(allText))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
print(allText[:50])

[['臺灣', '高雄', '地方法院', '刑事判決', '年度', '交簡上', '字', '第', '號', '上', '訴', '人', '即', '被', '告', '吳文良', '上列', '上訴人', '因', '過失傷害', '案件', '不服', '本院', '年度', '交', '簡字', '第', '號', '第一審', '簡易', '判決', '聲請', '簡易', '判決', '處刑', '案號', '年度', '調', '偵字', '第', '號', '提起', '上訴', '本院', '判決', '如下', '主', '文', '原', '判決', '撤銷', '吳文良', '犯', '過失傷害', '罪', '處', '拘役', '伍拾', '伍日', '如易科', '罰金', '以', '新臺幣', '壹仟', '元', '折算', '壹日', '事', '實', '一', '吳文良考', '領有', '普通', '重型', '機車', '駕駛執照', '於', '民國', '年', '月', '日', '時', '分許', '騎乘', '車牌號碼', '號', '普通', '重型', '機車', '沿', '高雄市', '鳳山', '區', '國光', '路由', '西往東', '方', '向右轉', '駛入', '五甲', '一路', '再', '沿', '五甲', '一路', '外側', '車道', '由北', '往南方', '向', '行駛', '時', '本應', '注意', '行車', '變換', '車道', '時', '應先', '顯示', '方向', '燈光', '且', '機車', '變換', '車道', '時', '應', '讓', '直行', '車', '先行', '並', '注意安全', '距離', '而', '依', '當時', '天候', '晴', '夜間', '有', '照明', '柏油路', '面', '乾燥', '無', '缺陷', '無', '障礙物', '視距', '良好', '客觀', '上及', '依', '其', '智識', '經驗', '能力', '並無', '不能', '注意', '之', '情事', '其竟', '疏', '未', '注意', '及', '此', '未先', '顯示'

In [42]:
f = open("corpus.txt",'w')
for d in allText:
    for t in d :
        f.write("%s " % str(t))

In [43]:
test_f = open("corpus.txt",'r')
print(test_f.read().split(" ")[:50])

['臺灣', '高雄', '地方法院', '刑事判決', '年度', '交簡上', '字', '第', '號', '上', '訴', '人', '即', '被', '告', '吳文良', '上列', '上訴人', '因', '過失傷害', '案件', '不服', '本院', '年度', '交', '簡字', '第', '號', '第一審', '簡易', '判決', '聲請', '簡易', '判決', '處刑', '案號', '年度', '調', '偵字', '第', '號', '提起', '上訴', '本院', '判決', '如下', '主', '文', '原', '判決']


In [44]:
model_name = "test"
model_text = "corpus.txt"
model_phrase = "corpus-phrase.txt"

#word2vec.word2phrase(model_text, model_phrase, verbose=True)

In [45]:
MIN_COUNT = 5
WORDVEC_DIM = 100
WINDOW = 5
NEGATIVE_SAMPLES = 5
ITERATIONS = 0
MODEL = 1
LEARNING_RATE = 0.025

In [59]:
model = word2vec.word2vec(
        train=model_text,
        output=model_name+".bin",
        size=WORDVEC_DIM,
#         min_count=MIN_COUNT,
#         window=WINDOW,
#         negative=NEGATIVE_SAMPLES,
#         alpha=LEARNING_RATE,
        verbose=True
        )

b'Starting training using file corpus.txt\n'b'100K\r200K\r300K\r400K\r500K\r600K\r700K\r800K\r900K\r1000K\r1100K\rVocab size: 10761\n'b'Words in train file: 1134529\n'b'\rAlpha: 0.025000  Progress: 0.18%  Words/thread/sec: 118.12k  \rAlpha: 0.025000  Progress: 0.37%  Words/thread/sec: 195.58k  \rAlpha: 0.024908  Progress: 0.56%  Words/thread/sec: 260.13k  \rAlpha: 0.024816  Progress: 0.92%  Words/thread/sec: 411.71k  \rAlpha: 0.024908  Progress: 0.73%  Words/thread/sec: 339.93k  \rAlpha: 0.024769  Progress: 1.11%  Words/thread/sec: 274.91k  \rAlpha: 0.024769  Progress: 1.29%  Words/thread/sec: 286.19k  \rAlpha: 0.024678  Progress: 1.47%  Words/thread/sec: 270.12k  \rAlpha: 0.024632  Progress: 1.66%  Words/thread/sec: 303.41k  \rAlpha: 0.024585  Progress: 1.84%  Words/thread/sec: 332.98k  \rAlpha: 0.024539  Progress: 2.03%  Words/thread/sec: 363.30k  \rAlpha: 0.024493  Progress: 2.21%  Words/thread/sec: 323.34k  \rAlpha: 0.024402  Progress: 2.57%  Words/thread/sec: 353.15k  \rAlpha: 0.0

In [47]:
model = word2vec.load(model_name+".bin")

In [48]:
model.vocab

array(['</s>', '之', '第', ..., "'以上',", "'罪',", "'刑法',"],
      dtype='<U78')

In [49]:
model.vocab[2]

'第'

In [50]:
model.vectors.shape

(10761, 100)

In [51]:
model["有期徒刑"]

array([ 0.07239011,  0.09366628,  0.16972454, -0.00931197,  0.12395632,
       -0.19150773, -0.13041866, -0.03810989, -0.08880974, -0.07976452,
        0.00482042,  0.05434619,  0.01900878, -0.11635373, -0.09463493,
       -0.15859351,  0.09638973, -0.06135924,  0.13578489, -0.1453874 ,
       -0.00300692,  0.17124109, -0.08015459, -0.05511848,  0.04777848,
       -0.13224675, -0.00291793, -0.15325673, -0.07347489,  0.16262679,
        0.02531261, -0.16898265, -0.23394135, -0.0449343 , -0.09939686,
       -0.03250756,  0.08329216, -0.04259343,  0.16475564, -0.12327896,
        0.00856924, -0.1854388 ,  0.1773452 , -0.00655401,  0.18351384,
       -0.00467337,  0.10533763, -0.05321394,  0.08184003,  0.01210071,
       -0.09794204, -0.0324799 , -0.0240034 , -0.08700526,  0.25882813,
       -0.1166276 ,  0.0074758 ,  0.03767037,  0.08629017, -0.01108739,
       -0.1138348 ,  0.07903541,  0.22663842,  0.14120325, -0.03805635,
       -0.00226601,  0.03229225,  0.02384017, -0.05415704, -0.02

In [52]:
indexes, metrics = model.cosine("有期徒刑")

In [53]:
model.vocab[indexes]

array(['徒刑', '有期', '徒', '拘役', '確定', '壹年', '刑', '累犯', '參年', '如易'],
      dtype='<U78')

In [54]:
model.generate_response(indexes, metrics).tolist()

[('徒刑', 0.6649880114258068),
 ('有期', 0.6118679607394641),
 ('徒', 0.5862106274359867),
 ('拘役', 0.575410877345326),
 ('確定', 0.5675754856592332),
 ('壹年', 0.5648508855298986),
 ('刑', 0.5540563180242921),
 ('累犯', 0.5398636595041085),
 ('參年', 0.5382921273800135),
 ('如易', 0.5378173740416841)]

In [56]:
cluster = word2vec.word2clusters(model_text, 'law-clusters.txt', 100, verbose=True)

b'Starting training using file corpus.txt\n'b'100K\r200K\r300K\r400K\r500K\r600K\r700K\r800K\r900K\r1000K\r1100K\rVocab size: 10761\n'b'Words in train file: 1134529\n'b'\rAlpha: 0.025000  Progress: 0.18%  Words/thread/sec: 315.93k  \rAlpha: 0.024954  Progress: 0.37%  Words/thread/sec: 347.06k  \rAlpha: 0.024909  Progress: 0.55%  Words/thread/sec: 507.60k  \rAlpha: 0.024862  Progress: 0.73%  Words/thread/sec: 360.19k  \rAlpha: 0.024817  Progress: 0.91%  Words/thread/sec: 409.10k  \rAlpha: 0.024771  Progress: 1.10%  Words/thread/sec: 389.07k  \rAlpha: 0.024724  Progress: 1.29%  Words/thread/sec: 429.31k  \rAlpha: 0.024678  Progress: 1.47%  Words/thread/sec: 371.56k  \rAlpha: 0.024632  Progress: 1.65%  Words/thread/sec: 395.37k  \rAlpha: 0.024587  Progress: 1.84%  Words/thread/sec: 414.21k  \rAlpha: 0.024540  Progress: 2.02%  Words/thread/sec: 403.17k  \rAlpha: 0.024495  Progress: 2.21%  Words/thread/sec: 414.75k  \rAlpha: 0.024448  Progress: 2.39%  Words/thread/sec: 432.04k  \rAlpha: 0.0

In [57]:
indexes, metrics = model.analogy(pos=['前科', '有期徒刑'], neg=['偷竊'], n=10)

In [58]:
model.generate_response(indexes, metrics).tolist()

[('表可佐', 0.3017701384972582),
 ('品之', 0.30104397340068645),
 ('另因', 0.2946925077471645),
 ('月確', 0.26975399976403325),
 ('第一級', 0.2591056074989331),
 ('素行', 0.2590724778940001),
 ('之肆', 0.2503906621433908),
 ('累犯', 0.24947359141030262),
 ('各經', 0.24672771516216424),
 ('條將', 0.24472585883485748)]